In [51]:
from pykrx import stock
import pykrx.website.krx.bond.wrap as wrap

import pandas as pd
import chartify
import numpy as np
from scipy.stats import norm

In [40]:
#KT&G
ticker = "033780"

market_ohlcv = stock.get_market_ohlcv_by_date("20100101", "20200708", ticker)
market_fundamental = stock.get_market_fundamental_by_date("20100101", "20200708", ticker)
treasury_yields_in_bond = wrap.KrxBond().get_treasury_yields_in_bond_index("20100101", "20200708")

df1 = market_ohlcv.reset_index()
df2 = market_fundamental.reset_index()
df3 = treasury_yields_in_bond.reset_index()


div = df2[['날짜', 'DIV']]

bond = df3[['지표수익률', '3년물']]
bond.columns = ['날짜', '3년물']
bond['날짜'] = pd.to_datetime(bond['날짜'], format='%Y-%m-%d', errors='raise')

tot = pd.merge(div, bond, on='날짜', how='left')
tot['국채시가배당률'] = tot['DIV'] + tot['3년물']

WARN: fromdate seems to be a holiday
- 2010-01-04 is used instead of 20100101
WARN: todate seems to be a holiday
- 2020-07-08 is used instead of 20200708


c:\users\iannam\pycharmprojects\financialanalysis\venv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [79]:
# 이동평균선
days=[3, 10, 30, 120]

for rolling in days:
    ma = 'ma{}'.format(rolling)
    df1[ma] = df1['종가'].rolling(rolling).mean()
    

ch = chartify.Chart(blank_labels=True, 
                    x_axis_type='datetime',
                    y_axis_type='linear')

ch.plot.area(
    data_frame=df1,
    x_column='날짜',
    y_column='ma3',
    second_y_column='ma120')
ch.style.color_palette.reset_palette_order()
    
ch.plot.line(
    data_frame=df1,
    x_column='날짜',
    y_column='종가'
)
ch.show()

In [87]:
print(tot.head(100))

ch = chartify.Chart(blank_labels=True, 
                    x_axis_type='datetime',
                    y_axis_type='linear',
                    second_y_axis=True)    
    
ch.set_subtitle(
        "국채시가배당률 = 시가배당률 / 3년만기국채수익률"
)

ch.plot.line(
    data_frame=df1,
    x_column='날짜',
    y_column='종가'
)

    
ch.second_axis.plot.line(
    data_frame=tot,
    x_column='날짜',
    y_column='국채시가배당률',
)
ch.second_axis.axes.set_yaxis_range(0, 10)
ch.show()

           날짜   DIV  3년물  국채시가배당률
0  2010-01-04  4.33  0.0     4.33
1  2010-01-05  4.38  0.0     4.38
2  2010-01-06  4.44  0.0     4.44
3  2010-01-07  4.47  0.0     4.47
4  2010-01-08  4.49  0.0     4.49
..        ...   ...  ...      ...
95 2010-05-20  4.74  0.0     4.74
96 2010-05-24  4.88  0.0     4.88
97 2010-05-25  4.96  0.0     4.96
98 2010-05-26  4.99  0.0     4.99
99 2010-05-27  4.90  0.0     4.90

[100 rows x 4 columns]
